# 环境初始化（修复中文显示）
import sys
import os
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 导入增强版回测引擎
from lib.backtest_enhanced import EnhancedBacktestEngine, validate_pnl_calculation
from lib.backtest import BacktestEngine
from lib.performance import PerformanceCalculator
from lib.risk import RiskAnalyzer
from configs.contract_specs import CONTRACT_SPECS, get_multiplier

# 设置pandas显示选项
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# 修复matplotlib中文显示问题
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']  # 使用DejaVu Sans字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
# 如果需要显示中文，可以用英文替代或使用支持的字体

# 设置绘图样式
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("Environment initialized successfully")
print("Using Enhanced Backtest Engine with stop-loss and validation")

## 第1部分：环境准备和数据加载

In [1]:
# 环境初始化
import sys
import os
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 自定义模块
from lib.backtest import BacktestEngine, calculate_spread_pnl
from lib.performance import PerformanceCalculator
from lib.risk import RiskAnalyzer
from configs.contract_specs import CONTRACT_SPECS, get_multiplier

# 设置pandas显示选项
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# 绘图设置
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境初始化完成")

环境初始化完成


In [2]:
# 加载所有数据
print("加载数据...")

# 1. 加载信号
signals_df = pd.read_parquet('../data/signals/kalman_signals_final.parquet')
print(f"✓ 信号数据: {len(signals_df)} 条")

# 2. 加载价格数据
price_data = {}
for symbol in CONTRACT_SPECS.keys():
    try:
        df = pd.read_parquet(f'../data/futures/{symbol}.parquet')
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        price_data[symbol] = df
    except:
        pass
print(f"✓ 价格数据: {len(price_data)} 个品种")

# 3. 回测参数
INITIAL_CAPITAL = 5000000
TRANSACTION_COST = 0.0002
print(f"✓ 初始资金: {INITIAL_CAPITAL:,.0f} 元")

加载数据...
✓ 信号数据: 3930 条
✓ 价格数据: 14 个品种
✓ 初始资金: 5,000,000 元


## 第2部分：执行回测并计算每笔交易细节

In [ ]:
# 使用增强版引擎执行回测（包含止损和验证）
enhanced_engine = EnhancedBacktestEngine(
    initial_capital=INITIAL_CAPITAL,
    stop_loss_rate=0.10,  # 10%止损
    transaction_cost_rate=TRANSACTION_COST
)

# 执行所有交易并进行验证
enhanced_trades = []
validation_reports = []

for trade in completed_trades:
    try:
        # 使用增强版引擎执行交易
        trade_result = enhanced_engine.execute_trade_with_stop_loss(
            trade['open_signal'], 
            trade['close_signal'], 
            price_data
        )
        enhanced_trades.append(trade_result)
        
        # 验证PnL计算
        validation = validate_pnl_calculation(trade_result)
        validation_reports.append(validation)
        
    except Exception as e:
        print(f"Error processing trade: {e}")
        continue

print(f"Enhanced backtest completed: {len(enhanced_trades)} trades")
print(f"Stop-loss triggered: {sum(t['stop_loss_triggered'] for t in enhanced_trades)} times")

# 转换为DataFrame
enhanced_df = pd.DataFrame(enhanced_trades)
print(f"\nData fields: {len(enhanced_df.columns)}")
print(f"Including: PnL validation, stop-loss status, z-scores")

In [ ]:
# 按配对展示每笔交易的详细信息（包含Z-score）
print("="*120)
print("DETAILED TRADES BY PAIR (with Z-scores and Stop-loss)")
print("="*120)

# 获取所有配对
pairs = enhanced_df['pair'].unique()

for pair in sorted(pairs):
    pair_trades = enhanced_df[enhanced_df['pair'] == pair].copy()
    
    print(f"\n{'='*60}")
    print(f"Pair: {pair}")
    print(f"Total trades: {len(pair_trades)}")
    print(f"Total PnL: {pair_trades['net_pnl'].sum():.2f}")
    print(f"Win rate: {(pair_trades['net_pnl'] > 0).sum() / len(pair_trades):.2%}")
    print(f"Stop-loss triggered: {pair_trades['stop_loss_triggered'].sum()} times")
    
    # 合约信息
    if len(pair_trades) > 0:
        first_trade = pair_trades.iloc[0]
        print(f"Contract multipliers: {first_trade['y_symbol']}={first_trade['y_multiplier']}, "
              f"{first_trade['x_symbol']}={first_trade['x_multiplier']}")
    
    print(f"{'='*60}")
    
    # 显示每笔交易
    display_cols = [
        'open_date', 'close_date', 'holding_days',
        'position_ratio', 'open_z_score', 'close_z_score',
        'y_pnl', 'x_pnl', 'net_pnl', 'return_pct',
        'stop_loss_triggered'
    ]
    
    # 格式化显示
    for idx, trade in pair_trades.iterrows():
        print(f"\nTrade #{idx+1}:")
        print(f"  Date: {trade['open_date'].date()} -> {trade['close_date'].date()} ({trade['holding_days']} days)")
        print(f"  Position ratio: {trade['position_ratio']}")
        print(f"  Z-score: Open={trade['open_z_score']:.2f}, Close={trade.get('close_z_score', 'N/A')}")
        print(f"  Price change: Y={trade['y_change']:.2f} ({trade['y_change_pct']:.2f}%), "
              f"X={trade['x_change']:.2f} ({trade['x_change_pct']:.2f}%)")
        print(f"  PnL breakdown: Y={trade['y_pnl']:.2f}, X={trade['x_pnl']:.2f}")
        print(f"  Total PnL: Gross={trade['final_pnl']:.2f}, Net={trade['net_pnl']:.2f}")
        print(f"  Return: {trade['return_pct']:.2f}%")
        if trade['stop_loss_triggered']:
            print(f"  ⚠️ STOP-LOSS TRIGGERED")
    
    # 小计
    print(f"\nSubtotal for {pair}:")
    print(f"  Y-leg total: {pair_trades['y_pnl'].sum():.2f}")
    print(f"  X-leg total: {pair_trades['x_pnl'].sum():.2f}")
    print(f"  Transaction costs: {pair_trades['transaction_cost'].sum():.2f}")
    print(f"  Net PnL: {pair_trades['net_pnl'].sum():.2f}")

In [ ]:
# PnL计算验证报告
print("\n" + "="*120)
print("PNL CALCULATION VALIDATION REPORT")
print("="*120)

# 统计验证结果
total_checks = 0
passed_checks = 0
failed_trades = []

for report in validation_reports:
    total_checks += report['total']
    passed_checks += report['passed']
    
    if not report['valid']:
        failed_trades.append(report['trade_id'])

print(f"\nValidation Summary:")
print(f"  Total trades validated: {len(validation_reports)}")
print(f"  Total checks performed: {total_checks}")
print(f"  Checks passed: {passed_checks} ({passed_checks/total_checks*100:.1f}%)")
print(f"  Trades with issues: {len(failed_trades)}")

if failed_trades:
    print(f"\nTrades requiring attention:")
    for trade_id in failed_trades[:10]:  # 只显示前10个
        print(f"  - {trade_id}")

# 显示几个验证示例
print("\n" + "-"*60)
print("Sample Validation Details:")
print("-"*60)

for i, report in enumerate(validation_reports[:3]):
    print(f"\nTrade: {report['trade_id']}")
    for check in report['checks']:
        print(f"  {check}")

# 验证统计
print("\n" + "-"*60)
print("Validation Statistics by Check Type:")
print("-"*60)

check_stats = {}
for report in validation_reports:
    for check in report['checks']:
        if '✓' in check:
            check_type = check.split('✓')[1].split(':')[0].strip()
            check_stats[check_type] = check_stats.get(check_type, 0) + 1

for check_type, count in sorted(check_stats.items(), key=lambda x: x[1], reverse=True):
    print(f"  {check_type}: {count} passed")

In [3]:
# 执行信号配对
engine = BacktestEngine(initial_capital=INITIAL_CAPITAL)
signals = signals_df.sort_values('date').to_dict('records')
completed_trades = engine.execute_signals(signals)
print(f"配对完成: {len(completed_trades)} 笔交易")

配对完成: 134 笔交易


In [4]:
# 计算每笔交易的详细收益
def calculate_detailed_trade_pnl(trade, price_data):
    """计算交易的详细盈亏分解"""
    open_signal = trade['open_signal']
    close_signal = trade['close_signal']
    pair = open_signal['pair']
    
    if '-' not in pair:
        return None
    
    y_symbol, x_symbol = pair.split('-')
    
    if y_symbol not in price_data or x_symbol not in price_data:
        return None
    
    try:
        open_date = pd.to_datetime(open_signal['date'])
        close_date = pd.to_datetime(close_signal['date'])
        
        # 价格
        y_open = price_data[y_symbol].loc[open_date, 'close']
        x_open = price_data[x_symbol].loc[open_date, 'close']
        y_close = price_data[y_symbol].loc[close_date, 'close']
        x_close = price_data[x_symbol].loc[close_date, 'close']
        
        # 仓位和乘数
        position_ratio = open_signal.get('position_ratio', '1:1')
        y_ratio, x_ratio = map(int, position_ratio.split(':'))
        y_multiplier = get_multiplier(y_symbol)
        x_multiplier = get_multiplier(x_symbol)
        
        # 价格变化
        y_price_change = y_close - y_open
        x_price_change = x_close - x_open
        y_price_change_pct = y_price_change / y_open * 100
        x_price_change_pct = x_price_change / x_open * 100
        
        # 分腿收益计算
        y_pnl = y_price_change * y_ratio * y_multiplier  # 做多Y
        x_pnl = -x_price_change * x_ratio * x_multiplier  # 做空X
        gross_pnl = y_pnl + x_pnl
        
        # 名义价值
        y_notional = y_open * y_ratio * y_multiplier
        x_notional = x_open * x_ratio * x_multiplier
        total_notional = y_notional + x_notional
        
        # 成本
        transaction_cost = total_notional * TRANSACTION_COST * 2
        net_pnl = gross_pnl - transaction_cost
        
        # 收益率
        gross_return = gross_pnl / total_notional * 100
        net_return = net_pnl / total_notional * 100
        
        # 保证金占用
        margin_required = total_notional * 0.12
        margin_return = net_pnl / margin_required * 100
        
        return {
            # 基础信息
            'trade_id': f"{pair}_{open_date.strftime('%Y%m%d')}",
            'pair': pair,
            'y_symbol': y_symbol,
            'x_symbol': x_symbol,
            'open_date': open_date,
            'close_date': close_date,
            'holding_days': (close_date - open_date).days,
            
            # 仓位信息
            'position_ratio': position_ratio,
            'y_ratio': y_ratio,
            'x_ratio': x_ratio,
            'y_multiplier': y_multiplier,
            'x_multiplier': x_multiplier,
            
            # 价格信息
            'y_open': y_open,
            'y_close': y_close,
            'y_change': y_price_change,
            'y_change_pct': y_price_change_pct,
            'x_open': x_open,
            'x_close': x_close,
            'x_change': x_price_change,
            'x_change_pct': x_price_change_pct,
            
            # 盈亏分解
            'y_pnl': y_pnl,
            'x_pnl': x_pnl,
            'gross_pnl': gross_pnl,
            'transaction_cost': transaction_cost,
            'net_pnl': net_pnl,
            
            # 名义价值
            'y_notional': y_notional,
            'x_notional': x_notional,
            'total_notional': total_notional,
            'margin_required': margin_required,
            
            # 收益率
            'gross_return': gross_return,
            'net_return': net_return,
            'margin_return': margin_return,
            
            # 信号信息
            'open_z_score': open_signal.get('z_score', 0),
            'close_z_score': close_signal.get('z_score', 0),
            'beta': open_signal.get('beta', 1.0)
        }
    except Exception as e:
        return None

# 计算所有交易
trades_detailed = []
for trade in completed_trades:
    result = calculate_detailed_trade_pnl(trade, price_data)
    if result:
        trades_detailed.append(result)

trades_df = pd.DataFrame(trades_detailed)
print(f"成功计算 {len(trades_df)} 笔交易的详细信息")
print(f"数据包含 {len(trades_df.columns)} 个字段")

成功计算 134 笔交易的详细信息
数据包含 35 个字段


## 第3部分：每笔交易的详细分析

In [5]:
# 展示所有交易的关键信息
print("="*100)
print("所有交易明细表")
print("="*100)

# 选择关键列展示
display_columns = [
    'trade_id', 'holding_days', 'position_ratio',
    'y_change_pct', 'x_change_pct',
    'y_pnl', 'x_pnl', 'gross_pnl', 'net_pnl',
    'net_return', 'margin_return'
]

# 按净收益排序
trades_display = trades_df[display_columns].sort_values('net_pnl', ascending=False)

# 添加颜色标记
def color_negative(val):
    color = 'red' if val < 0 else 'green'
    return f'color: {color}'

# 显示前20笔最赚钱的交易
print("\n📈 Top 20 最盈利交易:")
trades_display.head(20)

所有交易明细表

📈 Top 20 最盈利交易:


,trade_id,holding_days,position_ratio,y_change_pct,x_change_pct,y_pnl,x_pnl,gross_pnl,net_pnl,net_return,margin_return
106,AG0-AU0_20250422,6,4:3,-0.07,-6.18,-360.00,154140.00,153780.00,152586.12,5.11,42.60
31,AG0-AU0_20240520,15,4:3,-1.52,-3.90,-7500.00,67800.00,60300.00,59408.33,2.67,22.21
109,AG0-AU0_20250512,4,4:3,-1.87,-2.65,-9240.00,61440.00,52200.00,51075.14,1.82,15.14
17,NI0-SF0_20240430,22,3:4,9.97,3.66,42810.00,-5160.00,37650.00,37421.85,6.56,54.67
84,CU0-SS0_20250127,16,4:5,1.93,-1.28,29200.00,4250.00,33450.00,32712.38,1.77,14.78
117,AG0-AU0_20250606,4,4:3,0.42,-1.04,2220.00,24540.00,26760.00,25607.71,0.89,7.41
97,AG0-AU0_20250403,5,4:3,-7.28,-2.68,-36180.00,59460.00,23280.00,22194.29,0.82,6.81
95,NI0-SN0_20250402,5,1:1,-8.36,-10.01,-10820.00,29790.00,18970.00,18799.18,4.40,36.68
87,AG0-AU0_20250214,3,4:3,-2.88,-1.64,-14280.00,33900.00,19620.00,18595.82,0.73,6.05
74,CU0-SS0_20250108,6,4:5,1.41,0.92,21000.00,-3000.00,18000.00,17273.51,0.95,7.93


In [6]:
# 显示最亏损的交易
print("\n📉 Bottom 20 最亏损交易:")
trades_display.tail(20)


📉 Bottom 20 最亏损交易:


,trade_id,holding_days,position_ratio,y_change_pct,x_change_pct,y_pnl,x_pnl,gross_pnl,net_pnl,net_return,margin_return
129,CU0-SS0_20250725,5,4:5,-0.40,-0.84,-6400.00,2750.00,-3650.00,-4414.30,-0.23,-1.93
98,CU0-SS0_20250407,1,4:5,-0.38,-0.35,-5600.00,1125.00,-4475.00,-5193.22,-0.29,-2.41
58,NI0-SS0_20241009,5,4:5,-0.35,1.01,-1880.00,-3500.00,-5380.00,-5735.18,-0.65,-5.38
72,NI0-SF0_20241216,1,3:4,-1.83,-1.07,-6930.00,1360.00,-5570.00,-5772.82,-1.14,-9.49
34,AG0-AU0_20240605,1,3:2,3.34,1.54,11700.00,-17040.00,-5340.00,-5922.86,-0.41,-3.39
29,CU0-SS0_20240530,1,4:5,-0.43,-0.51,-7200.00,1875.00,-5325.00,-6134.73,-0.30,-2.53
15,RB0-SM0_20240422,25,1:1,1.25,23.60,460.00,-7940.00,-7480.00,-7508.14,-10.67,-88.94
75,AG0-AU0_20250116,1,4:3,-0.50,0.24,-2340.00,-4680.00,-7020.00,-7975.87,-0.33,-2.78
18,CU0-SN0_20240520,2,1:1,-1.65,0.18,-7250.00,-510.00,-7760.00,-8046.15,-1.12,-9.37
50,NI0-SS0_20240809,5,4:5,-2.80,-1.85,-14640.00,6375.00,-8265.00,-8612.64,-0.99,-8.26


In [7]:
# 识别异常交易
print("\n⚠️ 异常交易识别:")

# 1. 超高收益率交易（绝对值>10%）
high_return = trades_df[abs(trades_df['net_return']) > 10]
if len(high_return) > 0:
    print(f"\n1. 超高收益率交易 (|收益率| > 10%): {len(high_return)} 笔")
    print(high_return[['trade_id', 'net_return', 'net_pnl', 'holding_days']].sort_values('net_return'))

# 2. 超短期交易（持仓<=1天）
short_term = trades_df[trades_df['holding_days'] <= 1]
if len(short_term) > 0:
    print(f"\n2. 超短期交易 (持仓<=1天): {len(short_term)} 笔")
    print(short_term[['trade_id', 'holding_days', 'net_pnl', 'net_return']].sort_values('net_pnl', ascending=False))

# 3. 超长期交易（持仓>20天）
long_term = trades_df[trades_df['holding_days'] > 20]
if len(long_term) > 0:
    print(f"\n3. 超长期交易 (持仓>20天): {len(long_term)} 笔")
    print(long_term[['trade_id', 'holding_days', 'net_pnl', 'net_return']].sort_values('holding_days', ascending=False))

# 4. 巨额名义价值交易（>300万）
large_notional = trades_df[trades_df['total_notional'] > 3000000]
if len(large_notional) > 0:
    print(f"\n4. 巨额名义价值交易 (>300万): {len(large_notional)} 笔")
    print(large_notional[['trade_id', 'total_notional', 'margin_required', 'net_pnl']].sort_values('total_notional', ascending=False))


⚠️ 异常交易识别:

1. 超高收益率交易 (|收益率| > 10%): 1 笔
            trade_id  net_return  net_pnl  holding_days
15  RB0-SM0_20240422      -10.67 -7508.14            25

2. 超短期交易 (持仓<=1天): 36 笔
             trade_id  holding_days   net_pnl  net_return
93   AG0-AU0_20250401             1   5154.07        0.19
42   CU0-SS0_20240723             1   4311.97        0.23
26   NI0-SF0_20240528             1   3746.05        0.61
3    NI0-SF0_20240416             1   3666.17        0.69
78   CU0-SN0_20250205             1   2607.44        0.41
28   NI0-SS0_20240530             1   2485.91        0.26
5    NI0-SS0_20240416             1   2387.24        0.27
54   NI0-SS0_20240905             1   1922.68        0.23
41   PB0-ZN0_20240722             1   1239.89        0.58
44   NI0-SS0_20240731             1    663.26        0.08
68   CU0-SN0_20241127             1    417.47        0.07
90   NI0-SF0_20250310             1    282.61        0.05
40   PB0-ZN0_20240717             1    162.84        0.07
11   PB0

## 第4部分：配对层面的详细分析

In [8]:
# 每个配对的详细统计
pair_detailed_stats = trades_df.groupby('pair').agg({
    # 交易统计
    'net_pnl': ['sum', 'mean', 'std', 'min', 'max', 'count'],
    'gross_pnl': 'sum',
    'transaction_cost': 'sum',
    
    # 持仓统计
    'holding_days': ['mean', 'std', 'min', 'max'],
    
    # 收益率统计
    'net_return': ['mean', 'std', 'min', 'max'],
    'margin_return': 'mean',
    
    # 名义价值
    'total_notional': 'mean',
    'margin_required': 'mean',
    
    # 合约信息
    'y_multiplier': 'first',
    'x_multiplier': 'first',
    'position_ratio': 'first'
})

# 扁平化列名
pair_detailed_stats.columns = ['_'.join(col).strip() for col in pair_detailed_stats.columns]

# 计算额外指标
pair_detailed_stats['win_rate'] = trades_df.groupby('pair')['net_pnl'].apply(lambda x: (x > 0).sum() / len(x))
pair_detailed_stats['sharpe'] = pair_detailed_stats['net_return_mean'] / pair_detailed_stats['net_return_std'] * np.sqrt(252)
pair_detailed_stats['cost_ratio'] = pair_detailed_stats['transaction_cost_sum'] / pair_detailed_stats['gross_pnl_sum'].abs() * 100

# 排序
pair_detailed_stats = pair_detailed_stats.sort_values('net_pnl_sum', ascending=False)

print("="*100)
print("配对详细统计报告")
print("="*100)

# 显示关键指标
display_cols = [
    'net_pnl_sum', 'net_pnl_count', 'win_rate', 
    'net_pnl_mean', 'net_pnl_std',
    'net_return_mean', 'sharpe',
    'holding_days_mean', 'cost_ratio'
]

pair_summary = pair_detailed_stats[display_cols].round(2)
pair_summary.columns = ['总净收益', '交易次数', '胜率', '平均收益', '收益标准差', 
                        '平均收益率%', 'Sharpe', '平均持仓天数', '成本占比%']
pair_summary

配对详细统计报告


,总净收益,交易次数,胜率,平均收益,收益标准差,平均收益率%,Sharpe,平均持仓天数,成本占比%
pair,,,,,,,,,
AG0-AU0,313607.35,15,0.60,20907.16,42307.79,0.74,7.74,3.60,4.43
NI0-SF0,38964.39,17,0.59,2292.02,10622.83,0.43,3.64,4.76,8.41
NI0-SN0,19872.14,11,0.45,1806.56,6299.68,0.40,4.28,5.55,8.00
CU0-SN0,8692.08,13,0.38,668.62,5801.78,0.10,1.88,4.92,27.99
HC0-SF0,2664.40,11,0.45,242.22,1116.05,0.32,3.14,5.82,9.99
RB0-SF0,2599.52,13,0.69,199.96,671.22,0.26,4.25,4.31,11.58
PB0-ZN0,-2652.44,15,0.40,-176.83,1620.85,-0.10,-2.06,2.80,86.14
RB0-SM0,-7190.61,11,0.45,-653.69,2502.34,-0.85,-3.81,7.55,4.51
NI0-SS0,-9709.30,14,0.50,-693.52,8246.80,-0.05,-0.82,3.64,100.81


In [9]:
# 每个配对的所有交易明细
print("\n每个配对的交易明细:")
print("="*100)

for pair in pair_summary.index:
    pair_trades = trades_df[trades_df['pair'] == pair]
    
    print(f"\n{'='*50}")
    print(f"配对: {pair}")
    print(f"交易次数: {len(pair_trades)}, 总收益: {pair_trades['net_pnl'].sum():.2f}")
    print(f"胜率: {(pair_trades['net_pnl'] > 0).sum() / len(pair_trades):.2%}")
    print(f"合约乘数: {pair.split('-')[0]}={pair_trades['y_multiplier'].iloc[0]}, {pair.split('-')[1]}={pair_trades['x_multiplier'].iloc[0]}")
    print(f"{'='*50}")
    
    # 显示该配对的所有交易
    display_cols = ['open_date', 'close_date', 'holding_days', 'position_ratio',
                   'y_pnl', 'x_pnl', 'net_pnl', 'net_return']
    print(pair_trades[display_cols].to_string(index=False))
    
    # 统计汇总
    print(f"\n小计:")
    print(f"  Y腿总收益: {pair_trades['y_pnl'].sum():.2f}")
    print(f"  X腿总收益: {pair_trades['x_pnl'].sum():.2f}")
    print(f"  交易成本: {pair_trades['transaction_cost'].sum():.2f}")
    print(f"  净收益: {pair_trades['net_pnl'].sum():.2f}")


每个配对的交易明细:

配对: AG0-AU0
交易次数: 15, 总收益: 313607.35
胜率: 60.00%
合约乘数: AG0=15, AU0=1000
 open_date close_date  holding_days position_ratio     y_pnl     x_pnl   net_pnl  net_return
2024-05-20 2024-06-04            15            4:3  -7500.00  67800.00  59408.33        2.67
2024-06-05 2024-06-06             1            3:2  11700.00 -17040.00  -5922.86       -0.41
2024-07-25 2024-07-29             4            4:3   7020.00 -16680.00 -10500.10       -0.50
2024-08-06 2024-08-07             1            4:3   2820.00  -5640.00  -3650.78       -0.18
2024-09-02 2024-09-05             3            4:3  -7140.00  -3660.00 -11658.34       -0.54
2024-10-21 2024-10-22             1            4:3   -600.00  -2040.00  -3591.22       -0.15
2024-11-22 2024-11-25             3            4:3  -8820.00  26160.00  16399.25        0.70
2025-01-16 2025-01-17             1            4:3  -2340.00  -4680.00  -7975.87       -0.33
2025-02-11 2025-02-13             2            4:3   3360.00   3540.00   5885.7

## 第5部分：盈亏分解和归因分析

In [ ]:
# 累计收益曲线（英文标签）
trades_df_sorted = trades_df.sort_values('close_date').reset_index(drop=True)
trades_df_sorted['cumulative_pnl'] = trades_df_sorted['net_pnl'].cumsum()

plt.figure(figsize=(14, 6))
plt.plot(trades_df_sorted['close_date'], trades_df_sorted['cumulative_pnl'], 
         linewidth=2, label='Cumulative Net PnL')
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.title('Cumulative PnL Curve (with Real Multipliers)', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Cumulative PnL (CNY)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"Final cumulative PnL: {trades_df_sorted['cumulative_pnl'].iloc[-1]:,.2f} CNY")

In [ ]:
# 盈亏归因图表
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Y腿vs X腿贡献散点图
axes[0, 0].scatter(trades_df['y_pnl'], trades_df['x_pnl'], alpha=0.5)
axes[0, 0].axhline(y=0, color='r', linestyle='--', alpha=0.3)
axes[0, 0].axvline(x=0, color='r', linestyle='--', alpha=0.3)
axes[0, 0].set_xlabel('Y腿收益（做多）')
axes[0, 0].set_ylabel('X腿收益（做空）')
axes[0, 0].set_title('Y腿 vs X腿收益分布')
axes[0, 0].grid(True, alpha=0.3)

# 标注不同配对
for pair in trades_df['pair'].unique():
    pair_data = trades_df[trades_df['pair'] == pair]
    axes[0, 0].scatter(pair_data['y_pnl'], pair_data['x_pnl'], 
                      label=pair, alpha=0.6, s=30)

# 2. 各配对的Y/X腿贡献堆积图
pair_pnl = trades_df.groupby('pair')[['y_pnl', 'x_pnl']].sum().sort_values('y_pnl')
pair_pnl.plot(kind='barh', stacked=True, ax=axes[0, 1])
axes[0, 1].set_xlabel('收益（元）')
axes[0, 1].set_title('各配对Y/X腿收益贡献')
axes[0, 1].axvline(x=0, color='black', linestyle='-', linewidth=0.5)

# 3. 毛收益vs交易成本
axes[1, 0].scatter(trades_df['gross_pnl'], trades_df['transaction_cost'], alpha=0.5)
axes[1, 0].set_xlabel('毛收益')
axes[1, 0].set_ylabel('交易成本')
axes[1, 0].set_title('毛收益 vs 交易成本')
axes[1, 0].grid(True, alpha=0.3)

# 4. 成本占比分布
cost_ratio = trades_df['transaction_cost'] / trades_df['gross_pnl'].abs() * 100
axes[1, 1].hist(cost_ratio, bins=30, edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('成本占毛收益比例(%)')
axes[1, 1].set_ylabel('频次')
axes[1, 1].set_title('交易成本占比分布')
axes[1, 1].axvline(x=cost_ratio.mean(), color='r', linestyle='--', 
                   label=f'均值: {cost_ratio.mean():.1f}%')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 第6部分：时间序列分析

In [ ]:
# 按时间排序
trades_time = trades_df.sort_values('close_date').reset_index(drop=True)
trades_time['cumulative_pnl'] = trades_time['net_pnl'].cumsum()
trades_time['trade_number'] = range(1, len(trades_time) + 1)

# 创建时间序列图表
fig, axes = plt.subplots(3, 2, figsize=(16, 12))

# 1. 累计收益曲线（带每笔交易标记）
axes[0, 0].plot(trades_time['close_date'], trades_time['cumulative_pnl'], 
               linewidth=2, label='累计收益')
# 标记盈利和亏损交易
profit_trades = trades_time[trades_time['net_pnl'] > 0]
loss_trades = trades_time[trades_time['net_pnl'] < 0]
axes[0, 0].scatter(profit_trades['close_date'], profit_trades['cumulative_pnl'], 
                  color='green', s=20, alpha=0.5, label='盈利交易')
axes[0, 0].scatter(loss_trades['close_date'], loss_trades['cumulative_pnl'], 
                  color='red', s=20, alpha=0.5, label='亏损交易')
axes[0, 0].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[0, 0].set_xlabel('日期')
axes[0, 0].set_ylabel('累计收益（元）')
axes[0, 0].set_title('累计收益曲线（每笔交易标记）')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 每笔交易收益
colors = ['green' if x > 0 else 'red' for x in trades_time['net_pnl']]
axes[0, 1].bar(trades_time['trade_number'], trades_time['net_pnl'], color=colors, alpha=0.6)
axes[0, 1].set_xlabel('交易序号')
axes[0, 1].set_ylabel('净收益（元）')
axes[0, 1].set_title('每笔交易收益')
axes[0, 1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0, 1].grid(True, alpha=0.3)

# 3. 月度收益
trades_time['month'] = trades_time['close_date'].dt.to_period('M')
monthly_pnl = trades_time.groupby('month')['net_pnl'].sum()
monthly_pnl.plot(kind='bar', ax=axes[1, 0], color=['green' if x > 0 else 'red' for x in monthly_pnl])
axes[1, 0].set_xlabel('月份')
axes[1, 0].set_ylabel('净收益（元）')
axes[1, 0].set_title('月度收益分布')
axes[1, 0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1, 0].tick_params(axis='x', rotation=45)

# 4. 配对活跃度时间分布
pair_time_dist = trades_time.groupby(['month', 'pair']).size().unstack(fill_value=0)
pair_time_dist.plot(kind='area', stacked=True, ax=axes[1, 1], alpha=0.7)
axes[1, 1].set_xlabel('月份')
axes[1, 1].set_ylabel('交易次数')
axes[1, 1].set_title('各配对交易活跃度时间分布')
axes[1, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# 5. 滚动胜率（20笔交易窗口）
window = 20
trades_time['win'] = (trades_time['net_pnl'] > 0).astype(int)
trades_time['rolling_win_rate'] = trades_time['win'].rolling(window).mean()
axes[2, 0].plot(trades_time['trade_number'], trades_time['rolling_win_rate'], linewidth=2)
axes[2, 0].axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='50%胜率线')
axes[2, 0].set_xlabel('交易序号')
axes[2, 0].set_ylabel('胜率')
axes[2, 0].set_title(f'滚动胜率（{window}笔交易窗口）')
axes[2, 0].legend()
axes[2, 0].grid(True, alpha=0.3)

# 6. 滚动平均收益（20笔交易窗口）
trades_time['rolling_avg_pnl'] = trades_time['net_pnl'].rolling(window).mean()
axes[2, 1].plot(trades_time['trade_number'], trades_time['rolling_avg_pnl'], linewidth=2)
axes[2, 1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[2, 1].set_xlabel('交易序号')
axes[2, 1].set_ylabel('平均收益（元）')
axes[2, 1].set_title(f'滚动平均收益（{window}笔交易窗口）')
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 第7部分：风险分析

In [ ]:
# 风险指标计算
print("="*100)
print("风险分析报告")
print("="*100)

# 1. 连续亏损分析
def analyze_losing_streaks(trades_df):
    """分析连续亏损"""
    trades_sorted = trades_df.sort_values('close_date')
    
    streaks = []
    current_streak = []
    
    for _, trade in trades_sorted.iterrows():
        if trade['net_pnl'] < 0:
            current_streak.append(trade)
        else:
            if current_streak:
                streaks.append(current_streak)
                current_streak = []
    
    if current_streak:
        streaks.append(current_streak)
    
    return streaks

losing_streaks = analyze_losing_streaks(trades_df)

print("\n1. 连续亏损分析:")
if losing_streaks:
    max_streak = max(losing_streaks, key=len)
    max_loss_streak = max(losing_streaks, key=lambda x: sum(t['net_pnl'] for t in x))
    
    print(f"  最长连续亏损: {len(max_streak)} 笔")
    print(f"  最长连亏总额: {sum(t['net_pnl'] for t in max_streak):.2f} 元")
    print(f"  最大金额连亏: {len(max_loss_streak)} 笔")
    print(f"  最大连亏金额: {sum(t['net_pnl'] for t in max_loss_streak):.2f} 元")
    
    print("\n  连续亏损详情:")
    for i, streak in enumerate(losing_streaks):
        if len(streak) >= 3:  # 只显示3笔及以上的连亏
            print(f"    连亏{i+1}: {len(streak)}笔, 总亏损{sum(t['net_pnl'] for t in streak):.2f}元")

# 2. 回撤分析
trades_time = trades_df.sort_values('close_date').reset_index(drop=True)
trades_time['cumulative_pnl'] = trades_time['net_pnl'].cumsum()
trades_time['peak'] = trades_time['cumulative_pnl'].expanding().max()
trades_time['drawdown'] = trades_time['cumulative_pnl'] - trades_time['peak']
trades_time['drawdown_pct'] = trades_time['drawdown'] / (INITIAL_CAPITAL + trades_time['peak']) * 100

max_dd = trades_time['drawdown'].min()
max_dd_pct = trades_time['drawdown_pct'].min()
max_dd_date = trades_time.loc[trades_time['drawdown'].idxmin(), 'close_date']

print("\n2. 回撤分析:")
print(f"  最大回撤金额: {max_dd:.2f} 元")
print(f"  最大回撤百分比: {max_dd_pct:.2f}%")
print(f"  最大回撤发生日期: {max_dd_date.date()}")

# 计算回撤恢复时间
dd_periods = []
in_dd = False
dd_start = None

for i, row in trades_time.iterrows():
    if row['drawdown'] < 0 and not in_dd:
        in_dd = True
        dd_start = i
    elif row['drawdown'] == 0 and in_dd:
        in_dd = False
        dd_periods.append(i - dd_start)

if dd_periods:
    print(f"  平均回撤恢复期: {np.mean(dd_periods):.1f} 笔交易")
    print(f"  最长回撤恢复期: {max(dd_periods)} 笔交易")

# 3. 配对风险贡献
pair_risk = trades_df.groupby('pair').agg({
    'net_pnl': ['std', lambda x: (x < 0).sum()],
    'net_return': ['min', 'max', 'std']
})
pair_risk.columns = ['收益标准差', '亏损次数', '最大亏损率', '最大盈利率', '收益率标准差']

print("\n3. 配对风险贡献:")
print(pair_risk.sort_values('收益标准差', ascending=False))

In [ ]:
# 风险分布图表
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. 收益分布直方图
axes[0, 0].hist(trades_df['net_pnl'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[0, 0].axvline(x=trades_df['net_pnl'].mean(), color='g', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('净收益（元）')
axes[0, 0].set_ylabel('频次')
axes[0, 0].set_title('收益分布')

# 2. 收益率分布
axes[0, 1].hist(trades_df['net_return'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('净收益率（%）')
axes[0, 1].set_ylabel('频次')
axes[0, 1].set_title('收益率分布')

# 3. Q-Q图（检验正态性）
from scipy import stats
stats.probplot(trades_df['net_pnl'], dist="norm", plot=axes[0, 2])
axes[0, 2].set_title('Q-Q图（正态性检验）')

# 4. 回撤时间序列
axes[1, 0].fill_between(trades_time.index, 0, trades_time['drawdown'], 
                        where=trades_time['drawdown']<0, color='red', alpha=0.3)
axes[1, 0].plot(trades_time.index, trades_time['drawdown'], color='red', linewidth=1)
axes[1, 0].set_xlabel('交易序号')
axes[1, 0].set_ylabel('回撤（元）')
axes[1, 0].set_title('回撤时间序列')
axes[1, 0].grid(True, alpha=0.3)

# 5. 配对风险热力图
pair_risk_matrix = trades_df.pivot_table(
    index='pair', 
    values=['net_pnl', 'net_return'], 
    aggfunc=['std', 'min', 'max']
)
sns.heatmap(pair_risk_matrix.T, annot=True, fmt='.1f', cmap='RdYlGn_r', 
           center=0, ax=axes[1, 1])
axes[1, 1].set_title('配对风险指标热力图')

# 6. 持仓天数vs收益散点图
axes[1, 2].scatter(trades_df['holding_days'], trades_df['net_pnl'], alpha=0.5)
axes[1, 2].axhline(y=0, color='r', linestyle='--', alpha=0.3)
axes[1, 2].set_xlabel('持仓天数')
axes[1, 2].set_ylabel('净收益（元）')
axes[1, 2].set_title('持仓天数 vs 收益')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 第8部分：问题诊断和改进建议

In [ ]:
# 问题诊断
print("="*100)
print("问题诊断报告")
print("="*100)

# 1. 识别问题配对
print("\n1. 问题配对识别:")

# 亏损配对
loss_pairs = pair_summary[pair_summary['总净收益'] < 0]
if len(loss_pairs) > 0:
    print("\n  持续亏损配对:")
    for pair in loss_pairs.index:
        pair_data = trades_df[trades_df['pair'] == pair]
        print(f"    {pair}:")
        print(f"      总亏损: {loss_pairs.loc[pair, '总净收益']:.2f} 元")
        print(f"      胜率: {loss_pairs.loc[pair, '胜率']:.2%}")
        print(f"      交易次数: {loss_pairs.loc[pair, '交易次数']:.0f}")
        print(f"      合约乘数: {pair_data['y_multiplier'].iloc[0]} vs {pair_data['x_multiplier'].iloc[0]}")
        print(f"      建议: 考虑停止交易或调整策略参数")

# 低胜率配对
low_win_pairs = pair_summary[pair_summary['胜率'] < 0.4]
if len(low_win_pairs) > 0:
    print("\n  低胜率配对（<40%）:")
    for pair in low_win_pairs.index:
        print(f"    {pair}: 胜率={low_win_pairs.loc[pair, '胜率']:.2%}")

# 高成本配对
high_cost_pairs = pair_summary[pair_summary['成本占比%'] > 5]
if len(high_cost_pairs) > 0:
    print("\n  高成本配对（成本>5%）:")
    for pair in high_cost_pairs.index:
        print(f"    {pair}: 成本占比={high_cost_pairs.loc[pair, '成本占比%']:.1f}%")

# 2. 交易质量问题
print("\n2. 交易质量问题:")

# 超短期交易
short_trades = trades_df[trades_df['holding_days'] <= 1]
if len(short_trades) > 0:
    short_pnl = short_trades['net_pnl'].sum()
    print(f"\n  超短期交易（<=1天）:")
    print(f"    数量: {len(short_trades)} 笔 ({len(short_trades)/len(trades_df)*100:.1f}%)")
    print(f"    总收益: {short_pnl:.2f} 元")
    print(f"    平均收益: {short_trades['net_pnl'].mean():.2f} 元")
    print(f"    建议: 检查信号生成逻辑，可能存在过度交易")

# 巨额亏损交易
large_losses = trades_df[trades_df['net_pnl'] < -10000]
if len(large_losses) > 0:
    print(f"\n  巨额亏损交易（>1万）:")
    for _, trade in large_losses.iterrows():
        print(f"    {trade['trade_id']}: {trade['net_pnl']:.2f} 元")
        print(f"      收益率: {trade['net_return']:.2f}%")
        print(f"      名义价值: {trade['total_notional']:.0f} 元")

# 3. 风险集中度
print("\n3. 风险集中度分析:")

# 收益集中度
top3_pairs = pair_summary.head(3)
top3_pnl = top3_pairs['总净收益'].sum()
concentration = top3_pnl / trades_df['net_pnl'].sum() * 100

print(f"\n  Top3配对收益集中度: {concentration:.1f}%")
if concentration > 80:
    print(f"    警告: 收益过度集中，需要分散化")
    print(f"    主要依赖: {', '.join(top3_pairs.index)}")

# 名义价值集中度
pair_notional = trades_df.groupby('pair')['total_notional'].mean()
notional_std = pair_notional.std()
notional_mean = pair_notional.mean()
notional_cv = notional_std / notional_mean

print(f"\n  名义价值差异系数: {notional_cv:.2f}")
if notional_cv > 1:
    print(f"    警告: 各配对名义价值差异过大")
    print(f"    最大: {pair_notional.idxmax()} = {pair_notional.max():.0f}")
    print(f"    最小: {pair_notional.idxmin()} = {pair_notional.min():.0f}")

In [ ]:
# 改进建议汇总
print("\n" + "="*100)
print("策略改进建议")
print("="*100)

print("\n基于详细分析，提出以下改进建议:")

print("\n1. 配对调整:")
if len(loss_pairs) > 0:
    print(f"   - 剔除持续亏损配对: {', '.join(loss_pairs.index)}")
print(f"   - 重点关注高胜率配对: {', '.join(pair_summary[pair_summary['胜率'] > 0.6].index)}")

print("\n2. 风险控制:")
print(f"   - 设置单笔最大亏损限制: 建议 {abs(trades_df['net_pnl'].quantile(0.05)):.0f} 元")
print(f"   - 控制最大回撤: 当前 {max_dd_pct:.2f}%，建议控制在 5% 以内")

print("\n3. 仓位管理:")
print(f"   - 调整高名义价值配对的仓位")
high_notional = pair_notional[pair_notional > 2000000]
if len(high_notional) > 0:
    for pair, notional in high_notional.items():
        print(f"     {pair}: 当前 {notional:.0f} 元，建议减半")

print("\n4. 交易频率:")
if len(short_trades) / len(trades_df) > 0.2:
    print(f"   - 减少超短期交易，提高持仓时间阈值")

print("\n5. 成本优化:")
avg_cost_ratio = (trades_df['transaction_cost'] / trades_df['gross_pnl'].abs()).mean() * 100
print(f"   - 当前平均成本占比: {avg_cost_ratio:.2f}%")
if avg_cost_ratio > 3:
    print(f"   - 建议: 优化交易执行，降低滑点")

## 第9部分：完整交易明细表导出

In [ ]:
# 创建完整的交易明细表
print("生成完整交易明细表...")

# 选择所有重要字段
export_columns = [
    'trade_id', 'pair', 'open_date', 'close_date', 'holding_days',
    'y_symbol', 'x_symbol', 'position_ratio', 'y_multiplier', 'x_multiplier',
    'y_open', 'y_close', 'y_change', 'y_change_pct',
    'x_open', 'x_close', 'x_change', 'x_change_pct',
    'y_pnl', 'x_pnl', 'gross_pnl', 'transaction_cost', 'net_pnl',
    'y_notional', 'x_notional', 'total_notional', 'margin_required',
    'gross_return', 'net_return', 'margin_return',
    'open_z_score', 'close_z_score', 'beta'
]

# 创建导出DataFrame
export_df = trades_df[export_columns].copy()

# 添加累计收益
export_df = export_df.sort_values('close_date')
export_df['cumulative_pnl'] = export_df['net_pnl'].cumsum()
export_df['cumulative_return'] = export_df['cumulative_pnl'] / INITIAL_CAPITAL * 100

# 保存到CSV
output_dir = '../data/backtest_results'
os.makedirs(output_dir, exist_ok=True)

export_df.to_csv(f'{output_dir}/detailed_trades_analysis.csv', index=False)
print(f"✓ 完整交易明细已保存至: {output_dir}/detailed_trades_analysis.csv")

# 显示数据预览
print("\n数据预览（前10条）:")
export_df.head(10)

In [ ]:
# 生成配对统计表
pair_export = pair_detailed_stats.copy()
pair_export.to_csv(f'{output_dir}/pair_detailed_stats.csv')
print(f"✓ 配对详细统计已保存至: {output_dir}/pair_detailed_stats.csv")

# 生成问题交易清单
problem_trades = pd.concat([
    trades_df[trades_df['net_return'] < -10].assign(problem='超额亏损'),
    trades_df[trades_df['holding_days'] <= 1].assign(problem='超短期'),
    trades_df[trades_df['total_notional'] > 3000000].assign(problem='超大名义价值')
])

if len(problem_trades) > 0:
    problem_trades.to_csv(f'{output_dir}/problem_trades.csv', index=False)
    print(f"✓ 问题交易清单已保存至: {output_dir}/problem_trades.csv")
    print(f"  共识别 {len(problem_trades)} 笔问题交易")

## 第10部分：最终总结

In [ ]:
# 最终总结报告
print("="*100)
print("回测分析最终总结")
print("="*100)

print(f"\n回测期间: {trades_df['open_date'].min().date()} 至 {trades_df['close_date'].max().date()}")
print(f"总交易次数: {len(trades_df)}")
print(f"初始资金: {INITIAL_CAPITAL:,.0f} 元")
print(f"最终资金: {INITIAL_CAPITAL + trades_df['net_pnl'].sum():,.0f} 元")
print(f"净收益: {trades_df['net_pnl'].sum():,.2f} 元")
print(f"收益率: {trades_df['net_pnl'].sum()/INITIAL_CAPITAL:.2%}")

print("\n关键发现:")
print(f"1. 最佳配对: {pair_summary.index[0]}，贡献 {pair_summary.iloc[0]['总净收益']:.0f} 元")
print(f"2. 最差配对: {pair_summary.index[-1]}，亏损 {pair_summary.iloc[-1]['总净收益']:.0f} 元")
print(f"3. 平均胜率: {(trades_df['net_pnl'] > 0).mean():.2%}")
print(f"4. 最大单笔盈利: {trades_df['net_pnl'].max():.2f} 元")
print(f"5. 最大单笔亏损: {trades_df['net_pnl'].min():.2f} 元")
print(f"6. 平均持仓天数: {trades_df['holding_days'].mean():.1f} 天")
print(f"7. 交易成本占比: {trades_df['transaction_cost'].sum()/trades_df['gross_pnl'].sum()*100:.2f}%")

print("\n风险指标:")
print(f"1. 最大回撤: {max_dd:.2f} 元 ({max_dd_pct:.2f}%)")
print(f"2. 收益标准差: {trades_df['net_pnl'].std():.2f} 元")
print(f"3. 最长连续亏损: {len(max(losing_streaks, key=len)) if losing_streaks else 0} 笔")

print("\n主要问题:")
problems = []
if len(loss_pairs) > 0:
    problems.append(f"存在 {len(loss_pairs)} 个持续亏损配对")
if concentration > 80:
    problems.append(f"收益过度集中于少数配对 ({concentration:.1f}%)")
if len(short_trades) / len(trades_df) > 0.2:
    problems.append(f"超短期交易占比过高 ({len(short_trades)/len(trades_df)*100:.1f}%)")

for i, problem in enumerate(problems, 1):
    print(f"{i}. {problem}")

print("\n" + "="*100)
print("分析完成")
print("="*100)